In [1]:
import pandas
import numpy
from RexR import *
import _helpers
Rocket = RexR(datalocation = '/home/bramiozo/DEV/RexR/_data/genomic_data/data.pkl', seed = 3123, debug = True)
Rocket.load_probeset_data();

/usr/local/lib/python3.5/dist-packages/sklearn/qda.py:6: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


++++++++++++++++++++++++++++++ Firing up RexR! ++++++++++++++++++++++++++++++


In [2]:
##### LOAD IN DATA
###################

In [2]:
preds, class_model = Rocket.classify_treatment(model_type ='CART', 
                                              features = 'all', 
                                              grouping= 'mean', 
                                              n_splits = 10)

+ + + + + + + + + + + + + + + + + + + + + + + + + + + + + +  Prepping data
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - -  Grouping probesets
+ + + + + + + + + + + + + + + + + + + + + + + + + + + + + +  Creating X,y
++++++++++++++++++++++++++++++  RESULTS FOR CLASSIFICATION WITH GENOMIC DATA ++++++++++++++++++++++++++++++
MODEL: CART accuracy:  0.869565217391 +/-: 0.0143877336694
++++++++++++++++++++++++++++++  Report ++++++++++++++++++++++++++++++
             precision    recall  f1-score   support

          0       0.88      0.90      0.89        40
          1       0.86      0.83      0.84        29

avg / total       0.87      0.87      0.87        69

++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++  RESULTS FOR CLASSIFICATION INCLUDING PATIENT DATA ++++++++++++++++++++++++++++++
---------
MODEL: CART accuracy:  0.782608695652 +/-: 0.0172682809734
++++++++++++++++++++++++++++++  Report ++++++++++++++++++++++++++++++
             pre

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:2440: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [3]:
preds = [pred_[1]for pred_ in preds]
#len(Rocket.DATA_merged[Rocket.DATA_merged["array-batch"].isin(["cohort 1", "cohort 2", "JB", "IA", "ALL-10"])])
Results = Rocket.DATA_merged_processed
Results['pred'] = preds
print(Results[:311][['labnr patient', 'pred', 'Treatment risk group in ALL10']])

    labnr patient  pred Treatment risk group in ALL10
1             258  0.00                           NaN
2             531  0.00                           NaN
3             531  1.00                           NaN
4            2759  0.00                           NaN
5            2774  0.00                           NaN
6            1941  1.00                           NaN
7            1960  1.00                           NaN
8            2760  0.25                           NaN
9            2751  0.50                           NaN
10           2844  0.25                           NaN
11           1949  0.25                           NaN
12           2722  1.00                           NaN
13           2788  0.50                           NaN
14           2720  0.25                           NaN
15           2750  0.00                           NaN
16           2736  0.75                           NaN
17            634  1.00                           NaN
18            720  1.00     

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
nparr = numpy.array([[1,2,100],[1,1,1],[2,5,10],[2,4,2]])
nparr2= nparr.var()
print(nparr2)

727.576388889
